In [73]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import math
from math import log
import pprint

In [74]:
Cancer = load_breast_cancer()
X = Cancer.data
y = Cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(455, 30) (114, 30) (455,) (114,)


In [75]:
tree = DecisionTreeClassifier(random_state = 0)
tree.fit(X_train, y_train)
print("Accurary on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accurary on testing set: {:.3f}".format(tree.score(X_test, y_test)))

Accurary on training set: 1.000
Accurary on testing set: 0.939


In [76]:
Cancer = load_breast_cancer()
X = Cancer.data
y = Cancer.target
kf = KFold(n_splits = 5, random_state = 2001, shuffle = True)
for train_index, test_index in kf.split(X):
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]

In [77]:
tree = DecisionTreeClassifier(random_state = 0)
tree.fit(X_train, y_train)
print("Accurary on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accurary on testing set: {:.3f}".format(tree.score(X_test, y_test)))

Accurary on training set: 1.000
Accurary on testing set: 0.938


In [78]:
datasets = Cancer.data
labels = Cancer.feature_names
data_df = pd.DataFrame(datasets, columns=labels)
y = pd.DataFrame(Cancer.target)
df_cancer = pd.concat([data_df, y], axis=1)

In [79]:
df_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,0
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [80]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {
            'label:': self.label,
            'feature': self.feature,
            'tree': self.tree
        }

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)


class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p / data_length) * log(p / data_length, 2)
                    for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p) / data_length) * self.calc_ent(p)
                        for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        _, y_train, features = train_data.iloc[:, :
                                               -1], train_data.iloc[:,
                                                                    -1], train_data.columns[:
                                                                                            -1]
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(
                root=True,
                label=y_train.value_counts().sort_values(
                    ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(
                root=True,
                label=y_train.value_counts().sort_values(
                    ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(
            root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] ==
                                          f].drop([max_feature_name], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [84]:
dt = DTree()
tree = dt.fit(df_cancer)
tree

re': None, 'tree': {}}, 0.01899: {'label:': 1, 'feature': None, 'tree': {}}, 0.08271: {'label:': 0, 'feature': None, 'tree': {}}, 0.01404: {'label:': 1, 'feature': None, 'tree': {}}, 0.01115: {'label:': 1, 'feature': None, 'tree': {}}, 0.1021: {'label:': 0, 'feature': None, 'tree': {}}, 0.01921: {'label:': 1, 'feature': None, 'tree': {}}, 0.07507: {'label:': 0, 'feature': None, 'tree': {}}, 0.0218: {'label:': 1, 'feature': None, 'tree': {}}, 0.08923: {'label:': 0, 'feature': None, 'tree': {}}, 0.09113: {'label:': 0, 'feature': None, 'tree': {}}, 0.01117: {'label:': 1, 'feature': None, 'tree': {}}, 0.01257: {'label:': 1, 'feature': None, 'tree': {}}, 0.005592: {'label:': 1, 'feature': None, 'tree': {}}, 0.03731: {'label:': 1, 'feature': None, 'tree': {}}, 0.02623: {'label:': 1, 'feature': None, 'tree': {}}, 0.08353: {'label:': 0, 'feature': None, 'tree': {}}, 0.08543: {'label:': 0, 'feature': None, 'tree': {}}, 0.04391: {'label:': 1, 'feature': None, 'tree': {}}, 0.02008: {'label:': 1, 